In [1]:
import numpy as np

text = [

    "rnn", "learns", "from", "history",
    "rnn", "is", "powerful", "model",
    "rnn", "works", "on", "time-series",
    "rnn", "finds", "patterns", "sequences",
    "rnn", "has", "memory", "capabilities",
    "rnn", "is", "good", "for",
    "rnn", "predicts", "future", "events",
     "rnn", "is", "used", "widely",
    "rnn", "helps", "with", "sequences",
    "rnn", "can", "process", "data"
]



word_to_index = {word: i for i, word in enumerate(sorted(set(text)))}
index_to_word = {i: word for word, i in word_to_index.items()}
vocab_size = len(word_to_index)
seq_length = 3

def prepare_data(text, word_to_index, seq_length):
    sequences = []
    targets = []
    for i in range(len(text) - seq_length):
        seq = [word_to_index[word] for word in text[i:i+seq_length]]
        target = word_to_index[text[i+seq_length]]
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

def one_hot_encode(sequences, vocab_size):
    encoded = np.zeros((sequences.shape[0], sequences.shape[1], vocab_size))
    for i, seq in enumerate(sequences):
        for j, word_index in enumerate(seq):
            encoded[i, j, word_index] = 1
    return encoded

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def cross_entropy(pred, true_index):
    return -np.log(pred[true_index] + 1e-9)


class RNN:
    def __init__(self, input_size, hidden_size, output_size, lr=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.lr = lr

        self.Wx = np.random.randn(input_size, hidden_size) * 0.01
        self.Wh = np.random.randn(hidden_size, hidden_size) * 0.01
        self.Wy = np.random.randn(hidden_size, output_size) * 0.01
        self.bh = np.zeros((1, hidden_size))
        self.by = np.zeros((1, output_size))

    def forward(self, X):
        T = X.shape[0]
        self.h = np.zeros((T, self.hidden_size))
        self.o = np.zeros((T, self.output_size))
        for t in range(T):
            if t == 0:
                self.h[t] = np.tanh(X[t].dot(self.Wx) + self.bh)
            else:
                self.h[t] = np.tanh(X[t].dot(self.Wx) + self.h[t-1].dot(self.Wh) + self.bh)
            self.o[t] = self.h[t].dot(self.Wy) + self.by
        return self.o

    def predict(self, X):
        output = self.forward(X)
        probs = softmax(output[-1])
        return np.argmax(probs)

    def train(self, X_train, y_train, epochs=1000):
        for epoch in range(epochs):
            total_loss = 0
            for i in range(X_train.shape[0]):
                X = X_train[i]
                y_true = y_train[i]

                # FORWARD
                o = self.forward(X)
                probs = softmax(o[-1])
                loss = cross_entropy(probs, y_true)
                total_loss += loss

                # BACKWARD
                dWy = np.outer(self.h[-1], probs)
                dWy[:, y_true] -= self.h[-1]

                dby = probs.copy()
                dby[y_true] -= 1

                dh = probs.copy()
                dh[y_true] -= 1
                dh = dh.dot(self.Wy.T)

                dWx = np.zeros_like(self.Wx)
                dWh = np.zeros_like(self.Wh)
                dbh = np.zeros_like(self.bh)
                dh_next = np.zeros_like(dh)

                for t in reversed(range(seq_length)):
                    h_raw = (1 - self.h[t] ** 2) * (dh + dh_next)
                    dWx += np.outer(X[t], h_raw)
                    dbh += h_raw
                    if t > 0:
                        dWh += np.outer(self.h[t-1], h_raw)
                        dh_next = h_raw.dot(self.Wh.T)


                self.Wx -= self.lr * dWx
                self.Wh -= self.lr * dWh
                self.Wy -= self.lr * dWy
                self.bh -= self.lr * dbh
                self.by -= self.lr * dby.reshape(1, -1)

            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {total_loss:.4f}")


sequences, targets = prepare_data(text, word_to_index, seq_length)
sequences_one_hot = one_hot_encode(sequences, vocab_size)


model = RNN(input_size=vocab_size, hidden_size=16, output_size=vocab_size, lr=0.1)
model.train(sequences_one_hot, targets, epochs=1000)

test_input = sequences_one_hot[0]
pred_idx = model.predict(test_input)
print("Input:", text[0:3])
print("Expected:", text[3])
print("Predicted:", index_to_word[pred_idx])


## made by yousef ahmed 4221244 ##
## Rnn model ##

Epoch 0, Loss: 121.9636
Epoch 100, Loss: 1.0531
Epoch 200, Loss: 0.3241
Epoch 300, Loss: 0.1872
Epoch 400, Loss: 0.1302
Epoch 500, Loss: 0.0994
Epoch 600, Loss: 0.0803
Epoch 700, Loss: 0.0673
Epoch 800, Loss: 0.0579
Epoch 900, Loss: 0.0508
Input: ['rnn', 'learns', 'from']
Expected: history
Predicted: history
